# Ensembles

También conocidos como metaclasificadores

## Motivación:

Machine Learning "No free-lunch theorem":

- Distintos algoritmos funcionan correctamente en distintas situaciones

- A priori no es factible saber qué algoritmo funcionará mejor

- Por más que ajuste muy bien en el conjunto de test, aún puede estar fallando en algunos casos y podría existir otro algoritmo que a esos los ajuste mejor

A la hora de predecir, combinar distintas opiniones puede ser una excelente idea (como cuando vamos al médico) !

# Ejemplo

![](files/images/Ensembles_example.PNG)

# Formas de caracterización

### Utilizando distintos espacios de predictoras

![](files/images/Ensembles_type_1.PNG)

### Utilizando el mismo espacio de predictoras

![](files/images/Ensembles_type_2.PNG)

### Dependiendo de la forma de los clasificadores base

- Homogeneos: conjunto de clasificadores de un mismo tipo (Ejemplo: todas redes neuronales)
- Heterogeneos: conjunto de clasificadores de distintos tipos (Ejemplo: redes neuronales, k-NN, logistic regression, etc.)

### Dependiendo la estructura del ensemble

- Paralelos: Todos los clasificadores bases realizan una predicción y se combinan de alguna manera para obtener una única salida
- Seriales: Se consultan los clasificadores bases de forma serial, donde cada uno recibe los datos de entrada y la salida del clasificador previo
- Jerárquicos: Se establece una jerarquía y las salidas de los clasificadores base constituyen las entradas del metaclasificador

# Resumen parcial

- Tenemos clasificadores base que sirven para obtener distintas opiniones sobre cada caso
- Cada uno de ellos se ajusta mejor a una serie de casos y peor en otros
- Lo ideal es combinar modelos sencillos
- ** DEBE EXISTIR VARIABILIDAD EN LAS COMBINACIONES ! **

# Métodos básicos

# Fusión de etiquetas

In [10]:
# 5 clasificadores, 3 etiquetas posibles.
outputs = [[0,1,0], [1,0,0], [1,0,0], [0,1,0], [0,1,0]]

## Voto por mayoría:

In [8]:
import numpy as np
print('Resultados parciales:', np.sum(outputs, axis=0))
print('Etiqueta final:', np.sum(outputs, axis=0).argmax(axis=0))

Resultados parciales: [2 3 0]
Etiqueta final: 1


## Mayoría simple:

La clase debe tener por lo menos la mitad de los votos.

## Voto por mayoría con umbral:
La clase debe superar una cantidad mínima de votos, de lo contrario se utiliza una clase distinta de salida para indicar esta situación.

## Voto por mayoría ponderado:

In [23]:
classifiers_weigth = [2,1,3]
partial_sum = [np.array(o) * np.array(classifiers_weigth) for o in outputs]
print('Etiquetas ponderadas:', partial_sum)
print('Resultados parciales:', np.sum(partial_sum, axis=0))
print('Etiqueta final:', np.sum(partial_sum, axis=0).argmax(axis=0))

Etiquetas ponderadas: [array([0, 1, 0]), array([2, 0, 0]), array([2, 0, 0]), array([0, 1, 0]), array([0, 1, 0])]
Resultados parciales: [4 3 0]
Etiqueta final: 0


## Usando probabilidades:
- Media artimética
- Minimo
- Máximo
- Mediana
- Media ponderada
- ...

# Métodos avanzados

## Bagging: Bootstrap AGGregatING

- Es una metodología o forma de construir el metaclasificador (no un algoritmo en sí)
- Se crean L conjuntos de datos a partir del conjunto inicial, para entrenar L clasificadores
- Para predecir se combinan las salidas de los L clasificadores utilizando alguna técnica (generalmente voto por mayoría)
- Cada conjunto de datos se genera utilizando "boostrap" (muestreo con reemplazamiento): algunas instancias van a quedar repetidas y otras se eliminarán
- Tiene sentido con clasificadores base que sean inestables ! (pequeños cambios en los datos productes resultados diferentes)
- Si tengo muchos datos los conjuntos probablemente sean similares y el método pierde eficacia

### Procedimiento

Fase de entrenamiento:

1) Para k clasificadores base, generar k conjuntos de datos con muestras ** boostrap ** a partir del conjunto de datos original 

2) Entrenar los k clasificadores, cada uno con uno de los conjuntos

Fase de predicción:

1) Clasificar la instancia con los k clasificadores

2) Combinar las salidas y devolver un único resultado

# Random Forest

![](files/images/ensembles.png)

- Es un algoritmo que utiliza Bagging
- Ensemble homogeneo y paralelo: árboles como algoritmos base y creados de forma independiente
- Introduce aleatoriedad extra: cada nodo del árbol se genera seleccionado un subconjunto aleatorio dentro de los atributos disponibles en cada momento

Hyper-parámetros para definir:

- Cantidad de estimadores base
- Porcentaje de variables a utilizar en cada split
- Y todos los que teníamos para un árbol (profundidad máxima, cantidad de instancias mínimas en cada nodo hoja, etc.)

### Algunas variantes ...

- Pasting: cuando los conjuntos de datos se generan sin reemplazamiento y una cantidad de datos menor a la original.
- Random Subspaces: cuando los conjuntos de datos se generan sobre un subconjunto de variables de entrada.
- Random Patches: cuando lso conjuntos de datos se generan con las dos condiciones previas.

# Boosting

- Al igual que Bagging, es una metodología o forma de construir el metaclasificador (no un algoritmo en sí)
- Los clasificadores base se construyen uno después del otro de forma incremental
- La idea subyacente es que cada modelo se concentre más en las intancias donde el anterior falló
- Se arranca entrenando un estimador base con una muestra boostrap, pero a partir del segundo conjunto de datos la probabilidad de seleccionar a cada instancia está condicionada al error previo

## AdaBoost (ADApting BOOSTint)

- Es un algoritmo que utiliza Boosting
- Ensemble homogeneo y en serie: no está condicionado a un tipo en particular, pero el ensemble se compone de estimador de un mismo tipo, creados de a uno
- Los estimadores base deben ser lo más simple posibles (no tiene sentido poner uno muy bueno de entrada)

### Procedimiento

Fase de entrenamiento:

1) Inicializar los pesos de cada instancia en 1 / N (siendo N la cantidad de instancias)

2) Generar una muestra boostrap utilizando los pesos de las instancias

3) Entrenar un estimador con la muestra generada

4) Calcular el error y actualizar los pesos de las instancias de acuerdo a ese error (a mayor error mayor peso)

5) Repetir desde el paso 2 hasta que el error del modelo estimado esté por debajo de un umbral definido

Fase de predicción:

1) Utilizar todos los estimadores base entrenados en la fase de entrenamiento

2) combinar las salidas utilizando la técnica de voto por mayoría ponderado (donde el peso de cada estimador está relacionado con el error obtenido en la fase de entrenamiento)

## Gradient Boosting

- Es otra variante que utiliza Boosting
- Al igual que AdaBoost, también es homogeneo y en serie
- Se basa en ir ajustando estimadores base utilizando las salidas del modelo previo como variable a predecir
- Generalmente se utilizan árboles como estimadores base
- Es uno de los modelos que más fama está teniendo últimamente (suele dar muy buenos resultados sin tener que preprocesar demasiado los datos)
- xgboost es una de las librerías más utilizadas: tiene interfaces para varios lenguajes, está súper completa y optimizada !

### Ejemplo:

a) Vamos a entrenar a Franco para que prediga cuál va a ser la temperatura de mañana en base a la temperatura de hoy, el viento y la humedad. 

b) Cuando conocemos los errores de Franco a partir de una muestra, ahora le pedimos a Fisa que se aprenda los mismos ejemplos para predecir cuánto va a errar Franco en cada uno.

En esa instancia la salida del modelo es lo que diga Franco + lo que estime Fisa.

El algoritmo termina cuando el error es 0 y cae por debajo de un umbral definido.

# Bagging vs. Boosting

## Similitudes:

- Combinan estimadores base de un mismo tipo
- Buscamos perder un poco de interpretabilidad para ganar en performance
- Las salidas de combinan utilizando votos por mayoría (o alguna variante)
- Todos los estimadores base se generan a partir del mismo conjunto de entrenamiento

## Diferencias:
- Bagging permite paralelizar la construcción del ensemble mientras que Boosting no
- Bagging aisla cada modelo construido, mientras que en Boosting el conocimiento se "comparte"
- Boosting otorga pesos a los estimadores base mientras que ne Bagging todos cuenta por igual